In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


In [ ]:
df_train = pd.read_csv('../input/train.csv')


In [ ]:
df_train.head()


In [ ]:
df_train['diagnosis'].value_counts()/len(df_train)


In [ ]:
def load_dataset(path):
    eye_files = os.listdir(path)
    return eye_files


In [ ]:
train_files = load_dataset('../input/train_images')
test_files = load_dataset('../input/test_images')


In [ ]:
dis_classes = df_train['diagnosis'].unique()


In [ ]:
print('There are %d total disease categories' %len(dis_classes))
print('There are %s total eye images. \n' % len(np.hstack([train_files, test_files])))
print('There are %d training eye images. \n' % len(train_files))
print('There are %d test eye images. \n' % len(test_files))


In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob

train_files = np.array(glob("../input/train_images/*"))
test_files = np.array(glob("../input/test_images/*"))
img = cv2.imread(train_files[1])
plt.imshow(img)
plt.show()


In [ ]:
train_files[1]


In [ ]:
df_train[df_train.id_code == 'cd01672507c9']


In [ ]:
import random
for i in range(10):
    plt.figure(figsize=(10,10))
    i = random.choice(os.listdir('../input/train_images'))
    i_c = i.split('.')[0]
    #print(os.path.join('../input/train_images', i))
    img = cv2.imread(os.path.join('../input/train_images', i))
    print(i, df_train[df_train.id_code == i_c])
    plt.imshow(img)
    plt.show()


In [ ]:
def ed_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature


In [ ]:
gray = cv2.cvtColor(cv2.imread('../input/train_images/3e61703b5ab2.png'), cv2.COLOR_BGR2GRAY)


In [ ]:
image=cv2.imread('../input/train_images/3e61703b5ab2.png')
plt.imshow(image)


In [ ]:
bins= 8
def ed_histogram(image, mask=None):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([image], [0,1,2], None, [bins,bins,bins],[0,256,0,256,0,256,] )
    cv2.normalize(hist,hist)
    return hist.flatten()


In [ ]:
ed_histogram(image)


In [ ]:
dis_classes


In [ ]:
labels=[]
global_features=[]
for x in train_files:
    image = cv2.imread(x)
    
    x_c = x.split('.')[2].split('/')[3]
    current_label =  np.array(df_train.loc[df_train.id_code == x_c,'diagnosis'])
    labels.append(current_label)
    
    fv_hu_moments = ed_hu_moments(image)
    fv_histogram = ed_histogram(image)
    
    global_feature = np.hstack([fv_hu_moments,fv_histogram])
    global_features.append(global_feature)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
scaler = MinMaxScaler(feature_range=(0,1))
scaled_features=scaler.fit_transform(np.array(global_features))

le = LabelEncoder()
target = le.fit_transform(labels)

from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(multi_class='ovr')


In [ ]:
model1.fit(global_features,labels)


In [ ]:
model1.score(global_features, labels)


In [ ]:
id_cds=[]
type(id_cds)


In [ ]:
test_features = []
id_cds = []
for x in test_files:
    image = cv2.imread(x)
    
    x_c = x.split('.')[2].split('/')[3]
    id_cds.append(x_c)
    
    fv_hu_moments = ed_hu_moments(image)
    fv_histogram = ed_histogram(image)
    
    test_feature = np.hstack([fv_hu_moments,fv_histogram])
    test_features.append(test_feature)
    


In [ ]:
test_preds = model1.predict(test_features)


In [ ]:
combined_results = pd.DataFrame({'id_code': id_cds, 'diagnosis': test_preds})


In [ ]:
combined_results.head()


In [ ]:
combined_results.shape


In [ ]:
combined_results.to_csv("submission.csv",index=False)
